# **Запрос к API**

In [ ]:
import requests
import pandas as pd


api_url = "https://api.hh.ru"
url = f"{api_url}/employers"

access_token = 'Добавить токен полученный при регистрации'
headers = {'Authorization': f'Bearer {access_token}'}
params = {'text': "Альфа банк",'area' : 1}


response = requests.get(url, headers=headers, params = params)


if response.status_code == 200:
    data = response.json()
    table_data = []

    for employer in data['items']:
        table_data.append({
            'Название компании': employer['name'],
            'Идентификатор компании': employer['id'],})

    df = pd.DataFrame(table_data)
    df.to_csv('employers_description.csv', index=False)

    print("Сохранено в 'employers_description.csv'")
else:
    print(f"Ошибка: {response.status_code}")

Сохранено в 'employers_description.csv'


In [40]:
df

,Название компании,Идентификатор компании
0,Альфа-Банк,80
1,ФСЗС АО Альфа-Банк,5845941


# **Доработка поисковых запросов**

**Тест декоратора на скрипте поисковых запросов**

In [56]:
def split_area_decorator(func):
  """
  Декоратор для расширения шаблона templates, который возвращается функцией hh_search_templates().
  Преобразует список, который передается функцией get_area_code() в параметр 'area' в шаблон поисковых запросов.
  Доставая из этого списка код региона и зарплату.
  """
  def wrapper():
      templates = func()
      new_templates = []

      for template in templates:
          if isinstance(template['area'], list):
              for area in template['area']:
                new_template = template.copy()
                new_template['area'] = area[0]
                new_template['salary_to'] = area[1]
                new_templates.append(new_template)
          else:
            new_templates.append(template)

      return new_templates
  return wrapper


@split_area_decorator
def hh_search_templates() -> list[dict]:

  area_salary_lst_consultant = {
                                "Москва": [1, 70_000],
                                "Санкт-Петербург": [2, 80_000]
                                }
  area_salary_lst_dks = {
                          "Оренбург, Уфа, Чебоксары" : [[70, 99, 107], 55_000]
                        }

  def get_area_code(area_dict):
    area_codes = list(area_dict.values())
    return area_codes
  templates = [{'direction_code': "МП",
                'position_name': "Консультант",
                'area': get_area_code(area_salary_lst_consultant)},

               {'direction_code': "ДКС",
                'position_name': "Старший специалист",
                'area': get_area_code(area_salary_lst_dks)},

               {'direction_code': "ДОП",
                'position_name': "Ведущий специалист",
                'area': 1,
                'salary_to' : 35_000}
               ]

  return templates

In [57]:
hh_search_templates()

[{'direction_code': 'МП',
  'position_name': 'Консультант',
  'area': 1,
  'salary_to': 70000},
 {'direction_code': 'МП',
  'position_name': 'Консультант',
  'area': 2,
  'salary_to': 80000},
 {'direction_code': 'ДКС',
  'position_name': 'Старший специалист',
  'area': [70, 99, 107],
  'salary_to': 55000},
 {'direction_code': 'ДОП',
  'position_name': 'Ведущий специалист',
  'area': 1,
  'salary_to': 35000}]

In [58]:
search_templates = hh_search_templates()
resumes_id_list_all = []
count = 0
for i, template in enumerate(search_templates):
    print(i, template)


0 {'direction_code': 'МП', 'position_name': 'Консультант', 'area': 1, 'salary_to': 70000}
1 {'direction_code': 'МП', 'position_name': 'Консультант', 'area': 2, 'salary_to': 80000}
2 {'direction_code': 'ДКС', 'position_name': 'Старший специалист', 'area': [70, 99, 107], 'salary_to': 55000}
3 {'direction_code': 'ДОП', 'position_name': 'Ведущий специалист', 'area': 1, 'salary_to': 35000}
